## Install python Dash packages required to run the application.

In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


## Install python Juypter-Dash packages required to run the application.

In [2]:
pip install jupyter-dash

Note: you may need to restart the kernel to use updated packages.


## Install python Pandas packages required to run the application.

In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


## Import required libraries.

In [4]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/var/folders/1q/dqqrpzpx5jg604sl0wz8p9lw0000gn/T/ipykernel_8520/1855361594.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/1q/dqqrpzpx5jg604sl0wz8p9lw0000gn/T/ipykernel_8520/1855361594.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


## Read the airline data into pandas dataframe.

In [5]:
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv')

## First Five Row of the Table.

In [7]:
airline_data.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Shape of the Table.

In [8]:
airline_data.shape

(27000, 110)

# Create a dash application.
## Build dash app layout

In [9]:
# Create a dash application
app = dash.Dash(__name__)

# Build dash app layout
app.layout = html.Div(children=[ html.H1('Flight Delay Time Statistics', 
                                style={'textAlign': 'center', 'color': 'red',
                                'font-size': 30}),
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', 
                                type='number', style={'height':'35px', 'font-size': 30, 'border' : '1px solid red'}),], 
                                style={'font-size': 30, 'text-align' : 'center'}),
                                html.Br(),
                                html.Br(), 
                                # Segment 1
                                html.Div([
                                        html.Div(dcc.Graph(id='carrier-plot')),
                                        html.Div(dcc.Graph(id='weather-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 2
                                html.Div([
                                        html.Div(dcc.Graph(id='nas-plot')),
                                        html.Div(dcc.Graph(id='security-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 3
                                html.Div(dcc.Graph(id='late-plot'), style={'width':'100%'})
                                ])

# Compute_info Function
## This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

In [10]:
def compute_info(airline_data, entered_year):
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

# Callback Function
## Function that returns figures using the provided input year.

In [11]:
# Callback decorator
@app.callback( [
               Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'),
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')
               ],
               Input(component_id='input-year', component_property='value'))
# Computation to callback function and return graph
def get_graph(entered_year):

    # Compute required information for creating graph from the data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)

    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')

    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]

## Run the app

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_1m1671264250.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_1m1671264250.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_1m1671264250.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_1m1671264250.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET /_dash-component-suites/dash/deps/react@16.v2_7_1m1671264250.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_7_1m1671264250.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2023 19:53:15] "GET /_dash-component-suites/d